### These are my notes (no code) on learning about and working with GAMs and RNNs for this manuscript.

**05/23/20:**  

Received revisions on the MS.  Working on critiques.  
1. Reviewer asked to use objective method for computing the hyperparameters (epochs,batch_size,etc).  Am using sklearn.gridsearchv
2. Reviewer asked to use k-fold cross validation for scoring. This makes sense for model evaluation and hyperparameter choices, but not for ultimate model RMSD, because that is evaluated against the SBE43 and Licor.

** RNN with GridSearch CV **
GridSearchCV from sklearn uses k-fold cross validation.  I did k=5 cross validation test for epochs, batch_size, and dropouts.  Dropouts help remove complexity that reduces overfitting.  Results are found in ms1_methods/revisions/gridsearch_lstm_results.txt.  The best outcome was epochs=20, batch_size=80, dropouts = 0.4

** GAM with GridSearch CV **

The only two parameters in the GAM are the penalty coefficient and the degrees of freedom.  I've decided that it doesn't merit including that.


**01/10/20:**  Spent the past month working with underway data from the PIPERS cruise.  This is QMS ion currents generated by sipping from pCO2 equilibrator data aboard the NBP.  Heater cycling in the room was causing a periodic signal in the ion current.  Other factors also seemed to impact the ion current.  I implemented GAM and LSTM solutions to try to correct for this biasing.  

I found that **neither one was as effective at removing the bias**.  I think this is, in part, because I had no "in-situ calibration" where the periodic behavior could be observed in the absence of real environmental signals.

The LSTM solution had and RMSE = 27.73 and the GAM solution had RMSE = 26.76 showing how there is almost no difference in the quality of the solution.  This suggests that the quality of the solution is mostly limited by the detail or ability of the environmental correlates to reproduce the overall solution.

That last statement seems like it may be the principal takeaway from this article.  The in-situ calibration is necessary and helpful.

----- 
I recomputed the RMSE for latitudes 35 to 40 in SWIMS_GAM_LSTM_compare.py.  The results are GAM = 8.37 umol/kg and LSTM = 10.04 umol/kg.  




**12/13/19:**  Am interested to know if the LSTM can reconstruct the bias without needing an in-situ calibration.  For this, I will try loading a UMS file with the calibration data in it and train using data from that file, that does not include the in-situ cal. 

12/16 - am finding that the fit is significantly degraded when trying to fit to environmental data, rather than insitu cal bag data.

Found a section of the tow where ion current went way up - probably when instrument was being deploye or recovered. GAM and RNN were both waaay overestimating in this section.  This penalized GAM much more because it stayed high for a while.  Have opted to trim 100 points from beginning and end of each file.  

This dramatically reduced the RMSE for GAM and now they are nearly identical on average RMSE_GAM = 12.7, RMSE_LSTM = 12.9. 

If we compute RMSE for Lat 35-40 (the tow in the manuscript), find that GAM = 11.9 and LSTM = 11.4 (umol).

With a mean O2 concentration of 197 umol/L, this means the disagreement between SBE and SWIMS is about 6%.





**12/02/19:**  I'm writing the methods implementation of the LSTM. Some things are coming up as I go through it.

I chose N_input = 100 data points.  However, the interval of a tow is 700-800 data points.



**11/12/19**:  I had some concerns that the only way to achieve the good agreement we have was to "tune" to the Seabird DO sensor.  Therefore, I made a separate script that tunes O2 to solubility as with Ar, N2 and the others. This is captured in Z2_constr_O2_bysol.py.  It turns out, the standard approach works "ok", but the SWIMS overestimates the SBE35 DO transect, because DO was well below solubility equilibrium.  We can empirically adjust by fitting to a quantile of the data.  In our case, the 90% quantile worked well and this is what was fit to to adjust to the data. 

**11/04/19**:

Going through GAM code to make sure calculations are consistent with LSTM code.  Plotting histograms of the Mean-squared Error, etc.  

+ The X data in the GAM is now standardized (subtract mean, divide by std. dev).
+ Solution and MSE seem largely insensitive to the change in knots in the Spline:  Tried 3, 10, and 30 with little apparent difference.
+ Went back to using gamcalapply.py with the linear polynomial fit to water vapor.
+ 

LSTM:  Need to decide how many epochs to run for.  Maybe focus on fitting only for certain files.

Created Z2_epoch_test.py.   Using a subset of full data.  
verbose, batch_size = 1, 50. n_input = 100.

+ Epochs = 5, Batch = 32, RMSE = 15.1
+ Epochs = 5, RMSE = 16
+ Epochs = 10, RMSE = 16.9
+ Epochs = 15
+ Epochs = 20, RMSE = 18.97 (visibly degraded solution)
+ Epochs = 25, RMSE = 21.968
+ Epochs = 30, RMSE = 21.867

Clearly, overfitting is the biggest issue with the RNN and possibly with the GAM.  Best case scenario is Epochs = 5 and Batch = 32.

**Note:  Can I demonstrate that fitting can be done with a simple assumption of minimizing the difference between solubility equilibrium and gas concentration.   Current routine is actually relying on fit to the DO data.....**

**Need to choose a region of ZIPP2 where there is a lot of variation.  Cross the Gulf Stream and into the temperate coastal waters seems like a good strategy.**






**09/30/19**:

Somewhat chagrined to discover the LSTM neural network code appears to work better at simulating the long-term series in DO.  Am working now to evaluate the different input parameters, because these are consierably more opaque.

There are a lot of input parameters for an LSTM RNN; I will try to catalog them here.

Neurons:  This is the number of visible or hidden cells, I think.
Test/train split:  This is the percentage you use to test vs. train.
Stateful/Unstatful:  This is an algorithmic change that looks at previous network states to establish the initial condition.
Actiation function:  Am currently using 'tanh'
Loss estimator:  Currently using RMSE
optimizer:  Currently using 'adam'
Training Epochs:  

N_input:  Number of prior observations or number of data in a timestep
Batch_size:  How often to update the weights of the network? (For stateful transforms, batch_size must be a common factor of the dataset size.  I find that larger batch size speeds up processing, with some degradation in RMSE: at 64, RMSE = 0.24, and at 128 RMSE = 0.34.


Increasing the epochs does not notably improve the fit.   Will try decreasing the # samples per timestep.  This seems to be the most powerful tool.

I need a more detailed measure of "goodness of fit" . I think I will start making histograms of the 

Incremental change in the number of samples per.  

n_input, n_epoch, RMSE

+ 200, 5, 30.74
+ 300, 5, 33.73
+ **100, 5, 27.545**
+ 50, 5, 28.277
+ 10, 5, 30.16.   There is a progressive negative shift in the residuals as we shorten the # of inputs. This is based on a histrogram of the residuals.

**N_input = 100 seems to be the choice.**




**09/30/19**:

After spending further time investigating the backfit solution, I was able to find a compromise that seems to work well for both Ar and O2.  The order of fitting still matters and so, I update the partial residuals after a complete loop through all the explanatory variables, instead of after every new fit.  With this approach, there is additional stiffness in the fit to water vapor, so the spline fit works better overall than the linear polynomial.

I also implemented the GCV(\lambda) using notes and literature from Golub, Heath, and Wahaba 199?  I was able to implement their equation 1.4.
$$ V\left(\lambda \right) = \frac{1}{n}\left\| I-\textbf{A}(\lambda)y\right\|^2 \bigg/ \left[\frac{1}{n} Trace(I-\textbf{A}(\lambda))\right] $$
where $\textbf{A}$ is the projection or ?? matrix, and is computed as $\textbf{A}(\lambda) = \textbf{X}(\textbf{X}^T\textbf{X} + n\lambda I)^{-1}X^T $.  At first attempt, I did not have to use SVD to create a non-singular inverse of X, so it was easy to compute.  This may be necessary if I am computing it for later calculations.  Not sure if I can actually compute this on the full timeseries.  I don't see a local minimum in the GCV over a range of lambda from 1e-10 to 1e10, but I do see the limits where its effect dies off.  I will try to recompute for O2 with real oxygen data.  Chapter 4.6 around page 170. 

<img width=700, height=400, src="GCV_Mz40.png"></img>



**09/13/19**:

Happy to report that I think I have settled on a calibration protocol that is working for both Ar and O2.  However, I spent the whole week chasing some confusing aspects of the backfitting method. It appears the order that variables in X are introduced determines influence on the GAM.  Water vapor shows a strong dependence with O2 and has to be first in the GAM backfit or the fit is not as good.  Water vapor also works best with a 1st order polynomial, not a spline.

However, the appears not true for Argon; when I used m/z=18, I get large undersaturations in the surface ocean, which seems non-physical especially for ZIPP2 where the water surface is nearly 30C and should be oversaturated at those high temps. I tested multiple approachs to the fit, including using pyGAM's LinearGAM function and using backfit without the 1st order polynomial.  I also attempted to compute all partial residuals independently, while updating only after the first loop through all the columns in X (explanatory variables).  None of these achieve a superior fit, universally. 

The best fit is achieved with a order-1 polynomial applied to m/z18 only and excluding m/z18 from the fit to Argon.

Next I will try Nitrogen and CO2 before moving on to quantify the fit quality (ie, I need to compute the GCV and others).

**06/20/19**:

- Have now examined O2, N2, and Ar.  N2 and Ar show bad fit with mz18.  The effect is a too large vertical swing in these gases toward low saturation at depth that is not supported by the change in temperature.  Removing mz18 gives a much more realistic vertical change.

- The problem is that mz32 seems to produce a much better fit when mz18 is included as a linear polynomial.  I don't have a clear explanation why mz32 would be more sensitive to water vapor.

- Also, I note an overall decreasing trend in all ion currents.  This matches the decreasing trend in measured water temperature.  I don't know the best way to affect a good drift correction.  It seems the calibration factor alone did not happen often enough to remove all relevant baseline drift.

**06/10/19**:

- Have a complete DO timeseries for ZIPP2.  Ended up having to vary the CF in a linear fashion and also the GAM properties and IC0.  In the end, it was IC0 that produced the final tight fight tolerance.  This is captured in Z2_appcal.py.  


**05/31/19**:

- Working on Argon calibration today.  Finding that July 12 and July 14 calibrations have strange unexplained noise that Tim has observed.  They don't know where it comes from.


<img width=700, height=400, src="/Users/suelto/GDrive/research/ZIPP/cal_data/Argon40_calnoise.png"> cal_data/Argon40_calnoise.png</img>


- Am going to try to fit the GAM using July 15 and 17, then use the CFs from July 12 and 14.  The only test is solubility equilibrium.

- Have to make a decision on how to set the value of IC0.  I think its logical (based on Rachel Stanleys 2009 paper from Bermuda) that the deep water will be saturated to undersaturated and the surface water may be oversaturated as water heats faster than gas exchange can act.  The variation in Ar is bigger than we reasonably see.

**05/31/19**:

- Working on Argon calibration today.  Finding that July 12 and July 14 calibrations have strange unexplained noise that Tim has observed.  They don't know where it comes from.

<img width=700, height=400, src="../cal_data/Argon40_calnoise.png">cal_data/Argon40_calnoise.png</img>

- Am going to try to fit the GAM using July 15 and 17, then use the CFs from July 12 and 14.  The only test is solubility equilibrium.

- Have to make a decision on how to set the value of IC0.  I think its logical (based on Rachel Stanleys 2009 paper from Bermuda) that the deep water will be saturated to undersaturated and the surface water may be oversaturated as water heats faster than gas exchange can act.  The variation in Ar is bigger than we reasonably see.






**05/29/19**:

- Computed the CF for each calibration in ZIPP2_GAM_COMPUTE_CF.ipynb.  Finding quite a bit of drift over time. I suspect this may be associated with colder water (all ion currents appear to decline).  On July 12, mz32 is about 5e-11 and declines to 2.5e-11 by July 17.  
- Planning to transition from Jupyter to Spyder to apply the calibrations to e.g. all the files.

- Plotted the ctd data that Sam has saved in 'EN602_Loose/science/Triaxus/CTD/recal/EN602_Triaxus_CTD_recal.npz'. Again, I noticed some kind of time lag, compared to the UMS data.  Also, appears that the recalibration may not have impacted things that much.  Anyway, I have to load each CTD file by file and be very skeptical of the time stamp. This code was in Z2_appcal.py


~~~~ 
ctdt0 = np.datetime64('2017-07-08T05:46:16')
dat = np.load('/Users/suelto/GDrive/research/ZIPP/ZIPP2_EN602/EN602_Loose/science/Triaxus/CTD/recal/EN602_Triaxus_CTD_recal.npz')
ctdhdr = dat['ctd_names']
ctddat = dat['ctd_all']
ctdt = ctdt0+(ctddat[:,12]-ctddat[0,12]).astype('timedelta64')
~~~~~







**05/24/19**:

- Very excited to find it is not necessary to ratio the field data by the GAM reconstruction (y/yr), but it is possible to get a correction by subtracting!!   Y_correct = y - yr.

- The calibration factor must take into account that the ion current (IC) is not = 0 at [O2] = 0.  So, we use the equation for slope
$$ CF = \frac{x-x_0}{y-y_0} = \frac{[O2]_{cal}-0}{IC_{cal}-IC_0} $$
- The only problem is we did not measure IC0.  This can be done by stopping the pump and letting the water in the line become degassed.  Am trying to estimate from a previous event that Sam found, but it is not clear if one value of IC0 will hold for all calibrations.



**05/23/19**:

- Getting back to the GAM fitting. Current goal is to generate a usable ZIPP2 (then ZIPP1) data set and worry about the calibration paper later.
- Noticing some strange water vapor (mz18) behaviour below at 100 to 125 m. There is a strange symmetrical wiggle on the up and down cast.  This is not associated with the other ion currents and can't find similar features in turbo current, sample duty cycle, or sample temp.
- Need to make a thorough analysis of the calibration.  Also need to look back and figure out how to "zero" the ion currents.  Sam and I discussed this at some point.  It may be very hard to compute good calibrations without the "zero".




**05/01/18:**  
- Figured out how to implement the backfitting algorithm in Python and R.
- Backfitting algorithm lets me specify which model is applied to each data set.  Found that 
- Found that unfortunately all calibrations do not fall on a single line of mz32 vs mz18.
- Water temperature vs. mz18 shows a hysteresis that is not seen in mz32 vs mz18, so this is not a candidate for linear proportionality.

Notes on calibration files:

- MSData_7_12_2017 11_36.dfData:  Calibrating at 125 m by valve switching.  Definitely worth analyzing.
- MSData_7_12_2017 21_08.dfData: First good up/down calibration.
- MSData_7_14_2017 21_13.dfData: 
- MSData_7_15_2017 19_07.dfData: Notes say is a calibration on deck(explains why there is no ctd data).
- MSData_7_15_2017 22_44.dfData: Consistent.  
- MSData_7_17_2017 13_54.dfData: Bag calibration from 22:08 until